In [34]:
from TwitterSearch import TwitterSearchOrder, TwitterSearch, TwitterSearchException
import geopandas
from crawler_keywords import words
from sentiment_analysis import sentiment_analyzer
from database import tweetsDB
       

zones = geopandas.read_file('./geo_data/target_zones.geojson')
zone_code = zones.loc[0,:].zone
search_lat = zones.loc[0,:].center_latitude
search_long = zones.loc[0,:].center_longitude
search_radius = zones.loc[0,:].radius

db = tweetsDB()
sa = sentiment_analyzer.SentimentAnalyzer()

try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(words.get('vaccine'),True) # let's define all words we would like to have a look for
    tso.set_language('en') # we want to see German tweets only
    tso.set_include_entities(False) # and don't give us all those entity information
    tso.set_geocode(search_lat, search_long, int(search_radius))
    tso.set_count(10)

    # it's about time to create a TwitterSearch object with our secret tokens
    ts = TwitterSearch(
        consumer_key = "Ozup2OAf8pHZhha38AELtzswf",
        consumer_secret = "T0QCvEWdUm2PakSKDsho9usdvaPWZsUB0hhB2XE1JwgwCGp7wV",
        access_token = "1252083222189498368-BZUECSVoWd6IDSWGnETwH4krVS3AHh",
        access_token_secret = "y9UNO50rhFndcMPy4S4cY5qLEcZdz6NygWyl8t0ZY7H8s"
     )

     # this is where the fun actually starts :)
    for tweet in ts.search_tweets_iterable(tso):
        # Data to be written  
        dic ={  
          "id": tweet['id'],  
          "created_at": tweet['created_at'],
          "text": tweet['text'],  
          "zone_code": zone_code,
          "senti_score": sa.predict_sentiment(tweet['text'])
        } 
        #print(dic)
        #print( '@%s tweeted: %s' % ( tweet['user']['screen_name'], tweet['text'] ) )
        db.add_record(dic)

except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)
    raise

{'id': 1391316236844244992, 'created_at': 'Sun May 09 08:56:54 +0000 2021', 'text': 'RT @juliasilvertop: So Dan Andrews gets called a dictator for instigating lockdowns to save our lives but Morrison can keep us locked insid…', 'zone_code': 'Mel North', 'senti_score': (0.6874618, 'Positive')}


NameError: name 'db' is not defined

In [9]:
zones.loc[0,:].radius

7.214881296677225

In [30]:
zones = geopandas.read_file('./geo_data/target_zones.geojson')

In [31]:
zones

,zone,radius,center_longitude,center_latitude,geometry
0,Mel North,7.214881,144.936615,-37.754349,"POLYGON ((144.86281 -37.72129, 145.01112 -37.7..."
1,Mel West,7.649048,144.795628,-37.806474,"POLYGON ((144.73509 -37.75821, 144.85731 -37.7..."
2,Mel East,8.183116,145.127579,-37.839817,"POLYGON ((145.05781 -37.78969, 145.20201 -37.7..."
3,Mel Center,6.357621,144.957748,-37.829833,"POLYGON ((144.87791 -37.80488, 145.03035 -37.8..."
4,Mel SouthEast,7.889970,145.090060,-37.949026,"POLYGON ((145.00288 -37.90031, 145.14571 -37.9..."
5,Syd North,4.229974,151.195738,-33.818880,"POLYGON ((151.15127 -33.79360, 151.22817 -33.7..."
6,Syd Center,4.102252,151.199405,-33.885837,"POLYGON ((151.16912 -33.85663, 151.24877 -33.8..."
7,Syd East,4.714001,151.245184,-33.932253,"POLYGON ((151.25770 -33.87459, 151.20414 -33.9..."
8,Syd West,5.916964,151.089241,-33.890537,"POLYGON ((151.05239 -33.84722, 151.15402 -33.8..."
9,Syd Parramatta,7.832545,150.981634,-33.820676,"POLYGON ((150.93292 -33.76735, 151.05651 -33.7..."


In [18]:
from TwitterSearch import *
try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(['covid', 'vaccination']) # let's define all words we would like to have a look for
    tso.set_language('en') # we want to see German tweets only
    tso.set_include_entities(False) # and don't give us all those entity information
    tso.set_geocode(-37.840935, 144.946457, 100)
    tso.set_count(10)

    # it's about time to create a TwitterSearch object with our secret tokens
    ts = TwitterSearch(
        consumer_key = "Ozup2OAf8pHZhha38AELtzswf",
        consumer_secret = "T0QCvEWdUm2PakSKDsho9usdvaPWZsUB0hhB2XE1JwgwCGp7wV",
        access_token = "1252083222189498368-BZUECSVoWd6IDSWGnETwH4krVS3AHh",
        access_token_secret = "y9UNO50rhFndcMPy4S4cY5qLEcZdz6NygWyl8t0ZY7H8s"
     )

     # this is where the fun actually starts :)
    for tweet in ts.search_tweets_iterable(tso):
        print(tweet)
        print( '@%s tweeted: %s' % ( tweet['user']['screen_name'], tweet['text'] ) )

except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

{'created_at': 'Sun May 09 05:58:47 +0000 2021', 'id': 1391271412615159812, 'id_str': '1391271412615159812', 'text': 'RT @7NewsMelbourne: Australia’s border will stay shut until 2022 as the federal government targets COVID suppression, while the opposition…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1344744546937180160, 'id_str': '1344744546937180160', 'name': 'Grantapps77', 'screen_name': 'williamapss77', 'location': 'Bundaberg, Queensland', 'description': 'Jesus is my Lord, Saviour and King 👑.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 172, 'friends_count': 549, 'listed_count': 0, 'created_at': 'Thu Dec 31 20:37:48 +0

{'created_at': 'Sun May 09 00:36:48 +0000 2021', 'id': 1391190382252224512, 'id_str': '1391190382252224512', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 109535426, 'id_str': '109535426', 'name': 'virgo268', 'screen_name': 'DanaScully2', 'location': 'Adelaide Sth Australia', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 3211, 'friends_count': 5000, 'listed_count': 343, 'created_at': 'Fri Jan 29 11:25:34 +0000 2010', 'favourites_count': 269868, 'utc_offset': None,

{'created_at': 'Sat May 08 23:31:11 +0000 2021', 'id': 1391173866379091968, 'id_str': '1391173866379091968', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 252927646, 'id_str': '252927646', 'name': 'Morabeets', 'screen_name': 'morrisd3_dm', 'location': 'SFFS', 'description': "I've never met such a great group of people I've never met... recently titled by friend The Prince of Woke 🙂", 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 342, 'friends_count': 432, 'listed_count': 1, 'creat

{'created_at': 'Sat May 08 11:11:45 +0000 2021', 'id': 1390987784857350144, 'id_str': '1390987784857350144', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2169126492, 'id_str': '2169126492', 'name': 'Savitri Taylor', 'screen_name': 'SavitriTaylor', 'location': 'Melbourne, Australia', 'description': 'Legal academic; research area is #refugees & #asylum; views expressed my own; RTs, links, follows not endorsements.', 'url': 'https://t.co/0NwflZipen', 'entities': {'url': {'urls': [{'url': 'https://t.co/0NwflZipen', 'expanded_url': 'https://scho

{'created_at': 'Sat May 08 07:01:27 +0000 2021', 'id': 1390924791205416961, 'id_str': '1390924791205416961', 'text': "Here's a poster for a COVID vaccine drive at Bran Castle, Romania #vaccination #GetVaccinated #COVID19 https://t.co/tJ0uIFMvN3", 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 877869048809144321, 'id_str': '877869048809144321', 'name': 'christopher corneschi', 'screen_name': 'chrstphrcrnsch', 'location': 'Melbourne, Australia', 'description': 'graphic designer in training. always learning. he/him. runs @corbuldesign. drawings/illustrations: @cornyshiba.', 'url': 'https://t.co/QNYcZp17aV', 'entities': {'url': {'urls': [{'url': 'https://t.co/QNYcZp17aV', 'expanded_

...

{'created_at': 'Sat May 08 00:00:17 +0000 2021', 'id': 1390818802439360516, 'id_str': '1390818802439360516', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 260603193, 'id_str': '260603193', 'name': 'Sam Clemens 🕯🌈🐀⚖️🦋🕯#CovidisAirborne', 'screen_name': 'vvg26', 'location': 'Melbourne, Victoria', 'description': 'Continues to wear a mask, b/c it’s my job & surely yours? We’re all on the defence line. #CovidisAirborne ~ Fit For Purpose Quarantine, PPE, Ventilation', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': F

{'created_at': 'Fri May 07 23:46:44 +0000 2021', 'id': 1390815392457580545, 'id_str': '1390815392457580545', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2432770213, 'id_str': '2432770213', 'name': 'Dr. Amanda Caples', 'screen_name': 'Vic_LeadSci', 'location': 'Melbourne, Australia', 'description': 'Lead Scientist - aligning and connecting people to grow an effective, flexible innovation system in Victoria', 'url': 'https://t.co/zZ6l2pjgzQ', 'entities': {'url': {'urls': [{'url': 'https://t.co/zZ6l2pjgzQ', 'expanded_url': 'https://

{'created_at': 'Fri May 07 23:33:52 +0000 2021', 'id': 1390812154207502336, 'id_str': '1390812154207502336', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 20356005, 'id_str': '20356005', 'name': 'Peter Davidson 🇦🇺', 'screen_name': 'petedavo_au', 'location': 'Perth, Western Australia 🇦🇺', 'description': "#Australian & #Amyloidosis news.  RT's are only for OSINT sharing - See disclaimers on https://t.co/NRucefOAar     See @petedavo_world 🌏for other worldly shit.", 'url': 'https://t.co/u03oQpVFcR', 'entities': {'url': {'urls': [{'u

{'created_at': 'Fri May 07 23:26:02 +0000 2021', 'id': 1390810182028955653, 'id_str': '1390810182028955653', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 17954772, 'id_str': '17954772', 'name': 'Emily Gale', 'screen_name': 'EmilyGale', 'location': 'Australia', 'description': "Avid reader. Author. Next: Gisela Kaplan (Apr '21); Elsewhere Girls (May '21). Also The Other Side of Summer (2016), I Am Out With Lanterns (2018).", 'url': 'https://t.co/xyVZbtDTt7', 'entities': {'url': {'urls': [{'url': 'https://t.co/xyVZbtDTt7', 'expanded_url': 'htt

KeyboardInterrupt: 

In [29]:
import tweepy
import json
from sentiment_analysis import sentiment_analyzer
import geopandas
from crawler_keywords import words

consumer_key = "Ozup2OAf8pHZhha38AELtzswf"
consumer_secret = "T0QCvEWdUm2PakSKDsho9usdvaPWZsUB0hhB2XE1JwgwCGp7wV"
access_token = "1252083222189498368-BZUECSVoWd6IDSWGnETwH4krVS3AHh"
access_token_secret = "y9UNO50rhFndcMPy4S4cY5qLEcZdz6NygWyl8t0ZY7H8s"

zones = geopandas.read_file('./geo_data/target_zones.geojson')
zone_code = zones.loc[0,:].zone
zone_location=zones.loc[0,:].geometry.bounds
GEOBOX_MELBOURNE = [144.877548, -37.851203, 145.031356, -37.729655]  # https://boundingbox.klokantech.com/
GEOBOX_AUS = [106.8, -41.9, 154.9, -11.5]

# connect twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

# connect database
#sa = sentiment_analyzer.SentimentAnalyzer()

class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        if(status.place is not None):
            print("-----------new tweet found----------")
            #print("created at:",status.created_at)
            #print(status.text)
            #print("-------Saving to the database-----------")
            if any(x in status.text for x in words.get('vaccine')):
                senti_score = sa.predict_sentiment(status.text)
                db.add_record(status._json)
            #print("-------finished-----------")
            #print("-----------------------------------------------")

    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            # returning False in on_error disconnects the stream
            return False

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth, listener=myStreamListener)
myStream.filter(locations=zone_location, languages=["en"])

Error during authentication


SSLError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/filter.json?delimited=length (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)'),))